In [ ]:
import pandas as pd
import numpy as np
import re
from fuzzywuzzy import fuzz, process
import os
import json
import logging
import csv
from typing import Dict, List, Tuple, Union, Optional

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

def normalize_building_name(name: str) -> str:
    """Normalize building names to facilitate matching."""
    
    """Normalize building names to facilitate matching."""
    if name is None:
        return ""
    
    # Ensure name is a string
    try:
        name = str(name)
    except:
        return ""
    
    # Convert to lowercase
    name = name.lower()
    
    # Replace common abbreviations with standardized forms
    replacements = {
        'bldg': 'building', 
        'ctr': 'center', 
        'centre': 'center',
        'plz': 'plaza', 
        'twr': 'tower', 
        'indl': 'industrial',
        'comm': 'commercial', 
        'ltd': 'limited', 
        'hse': 'house',
        'intl': 'international',
        'dev': 'development',
        'fty': 'factory',
        'gdn': 'garden',
        'rd': 'road',
        'st': 'street',
        'ave': 'avenue'
    }
    
    # Replace abbreviations with full words
    for old, new in replacements.items():
        name = re.sub(r'\b' + old + r'\b', new, name)
    
    # Remove punctuation and normalize whitespace
    name = re.sub(r'[^\w\s]', ' ', name)
    name = re.sub(r'\s+', ' ', name).strip()
    
    return name

def normalize_address(address: str) -> str:
    """Normalize addresses to facilitate matching."""
    if address is None:
        return ""
    
    # Ensure address is a string
    try:
        address = str(address)
    except:
        return ""
    
    # Convert to lowercase
    address = address.lower()
    
    # Standardize road/street/avenue abbreviations
    replacements = {
        'rd': 'road',
        'rd.': 'road',
        'st': 'street',
        'st.': 'street',
        'ave': 'avenue',
        'ave.': 'avenue'
    }
    
    # Replace abbreviations with full words
    for old, new in replacements.items():
        address = re.sub(r'\b' + old + r'\b', new, address)
    
    # Normalize special characters and whitespace
    address = re.sub(r'[^\w\s]', ' ', address)
    address = re.sub(r'\s+', ' ', address).strip()
    
    return address

def extract_year(year_string: str) -> Optional[int]:
    """Extract year from various string formats."""
    if not isinstance(year_string, str):
        return None
    
    # Try to find a 4-digit year
    year_match = re.search(r'(\d{4})', year_string)
    if year_match:
        return int(year_match.group(1))
    
    return None

def load_csv_with_fallbacks(file_path, **kwargs):
    """Load CSV with multiple fallback methods if errors occur."""
    try:
        # First attempt - standard loading
        return pd.read_csv(file_path, **kwargs)
    except pd.errors.ParserError as e:
        logger.warning(f"Parser error with {file_path}: {str(e)}, trying with on_bad_lines='skip'")
        try:
            # Second attempt - skip bad lines
            return pd.read_csv(file_path, on_bad_lines='skip', **kwargs)
        except Exception as e2:
            logger.warning(f"Failed with on_bad_lines: {str(e2)}, trying with different quoting")
            try:
                # Third attempt - adjust quoting
                return pd.read_csv(file_path, quoting=csv.QUOTE_NONE, escapechar='\\', **kwargs)
            except Exception as e3:
                logger.warning(f"All parsing methods failed: {str(e3)}, trying with engine='python'")
                try:
                    # Fourth attempt - use Python engine
                    return pd.read_csv(file_path, engine='python', **kwargs)
                except Exception as e4:
                    logger.error(f"All CSV parsing methods failed: {str(e4)}")
                    # Create an empty DataFrame with the same structure
                    return pd.DataFrame()

def preprocess_csv_file(input_path, output_path=None):
    """Manually preprocess a problematic CSV file to fix inconsistent columns."""
    if output_path is None:
        output_path = input_path + ".fixed.csv"
    
    # Read the file to determine the expected number of columns
    with open(input_path, 'r', encoding='utf-8', errors='ignore') as f:
        first_line = f.readline().strip()
        expected_columns = first_line.count(',') + 1
        logger.info(f"Expected columns in CSV: {expected_columns}")
    
    # Process the file line by line
    with open(input_path, 'r', encoding='utf-8', errors='ignore') as infile, \
         open(output_path, 'w', encoding='utf-8', newline='') as outfile:
        
        writer = csv.writer(outfile)
        problem_lines = 0
        
        for i, line in enumerate(infile, 1):
            # Skip empty lines
            if not line.strip():
                continue
                
            # Split the line by commas not enclosed in quotes
            fields = []
            in_quotes = False
            current_field = ''
            
            for char in line:
                if char == '"':
                    in_quotes = not in_quotes
                    current_field += char
                elif char == ',' and not in_quotes:
                    fields.append(current_field)
                    current_field = ''
                else:
                    current_field += char
            
            # Add the last field
            fields.append(current_field)
            
            # Check if we have the right number of columns
            if len(fields) != expected_columns:
                problem_lines += 1
                logger.warning(f"Line {i}: Found {len(fields)} fields, expected {expected_columns}")
                
                if len(fields) > expected_columns:
                    # Combine extra fields into the last expected field
                    combined = fields[:expected_columns-1]
                    combined.append(','.join(fields[expected_columns-1:]))
                    fields = combined
                else:
                    # Pad with empty fields if fewer columns
                    fields.extend([''] * (expected_columns - len(fields)))
            
            writer.writerow(fields)
    
    logger.info(f"Preprocessing complete. Fixed {problem_lines} problem lines. Output saved to {output_path}")
    return output_path

def calculate_similarity(building1: Dict, building2: Dict) -> float:
    """Calculate similarity score between two buildings with improved type handling."""
    # Calculate name similarity using token sort ratio (handles word order differences)
    name_similarity = fuzz.token_sort_ratio(
        str(building1.get('normalized_name', '')), 
        str(building2.get('normalized_name', ''))
    )
    
    # Calculate name similarity using token set ratio (handles partial matches)
    name_set_similarity = fuzz.token_set_ratio(
        str(building1.get('normalized_name', '')), 
        str(building2.get('normalized_name', ''))
    )
    
    # Get the best name similarity score
    best_name_similarity = max(name_similarity, name_set_similarity)
    
    # Compare address if available
    address_similarity = 0
    if building1.get('address') is not None and building2.get('address') is not None:
        address1 = normalize_address(str(building1.get('address', '')))
        address2 = normalize_address(str(building2.get('address', '')))
        address_similarity = fuzz.token_sort_ratio(address1, address2)
    
    # Check district match
    district_match = 0
    if building1.get('district') and building2.get('district'):
        # Convert to string to ensure type consistency
        if str(building1.get('district', '')).lower() == str(building2.get('district', '')).lower():
            district_match = 100
    
    # Extract address numbers and check for matches
    address_number_match = 0
    try:
        addr1_nums = re.findall(r'\d+', str(building1.get('address', '')))
        addr2_nums = re.findall(r'\d+', str(building2.get('address', '')))
        if addr1_nums and addr2_nums:
            common_numbers = set(addr1_nums) & set(addr2_nums)
            if common_numbers:
                address_number_match = 100
    except Exception as e:
        logger.warning(f"Error extracting address numbers: {str(e)}")
        address_number_match = 0
    
    # Calculate weighted score
    weights = {
        'name': 0.6,
        'address': 0.2,
        'district': 0.1,
        'address_number': 0.1
    }
    
    weighted_score = (
        best_name_similarity * weights['name'] +
        address_similarity * weights['address'] +
        district_match * weights['district'] +
        address_number_match * weights['address_number']
    )
    
    return weighted_score


def create_consolidated_database(leasinghub_df=None, centanet_df=None, midland_df=None, match_threshold=75):
    """Create a consolidated database of buildings from multiple sources."""
    consolidated = {}
    
    # Process LeasingHub data first (assumes most complete source with grade info)
    if leasinghub_df is not None and not leasinghub_df.empty:
        logger.info(f"Processing {len(leasinghub_df)} buildings from LeasingHub")
        
        # Normalize LeasingHub data
        for idx, row in leasinghub_df.iterrows():
            key = f"LH_{row.get('ID', idx)}"
            
            year_built = None
            if pd.notna(row.get('Year Built')):
                try:
                    year_built = int(row.get('Year Built'))
                except (ValueError, TypeError):
                    year_built = extract_year(str(row.get('Year Built')))
            
            consolidated[key] = {
                'source': 'leasinghub',
                'source_id': row.get('ID'),
                'original_key': row.get('ID'),  # Keep original key for verification
                'building_name': row.get('Name', ""),
                'normalized_name': normalize_building_name(row.get('Name', "")),
                'address': row.get('Street Name', ""),
                'district': row.get('District Name', ""),
                'grade': row.get('Grade'),
                'year_built': year_built,
                'original_record': row.to_dict(),  # Store original record for eyeball check
                'matches': {}  # Will store matches to other sources
            }
    
    # Process Midland data
    if midland_df is not None and not midland_df.empty:
        logger.info(f"Processing {len(midland_df)} buildings from Midland")
        
        # Store unmatched records for later addition
        unmatched_midland = []
        
        for idx, row in midland_df.iterrows():
            # Extract building ID from the URL if available
            building_id = None
            if 'Detail URL' in row:
                url_match = re.search(r'B\d+', row.get('Detail URL', ''))
                if url_match:
                    building_id = url_match.group(0)
            
            midland_data = {
                'source': 'midland',
                'source_id': building_id,
                'original_key': building_id,  # Keep original key for verification
                'building_name': row.get('Building Name', ""),
                'normalized_name': normalize_building_name(row.get('Building Name', "")),
                'address': row.get('Address', ""),
                'original_record': row.to_dict(),  # Store original record
                'matches': {}
            }
            
            # Try to match to existing buildings from LeasingHub
            matched = False
            best_match_key = None
            best_match_score = 0
            
            for cons_key, cons_data in consolidated.items():
                match_score = calculate_similarity(midland_data, cons_data)
                if match_score >= match_threshold and match_score > best_match_score:
                    best_match_key = cons_key
                    best_match_score = match_score
            
            if best_match_key:
                # Add as a match to existing record
                consolidated[best_match_key]['matches']['midland'] = {
                    'id': midland_data['source_id'],
                    'original_key': building_id,  # Store original key
                    'score': best_match_score,
                    'building_name': midland_data['building_name'],
                    'address': midland_data['address'],
                    'original_record': row.to_dict()  # Add original record to match info
                }
                matched = True
            
            if not matched:
                unmatched_midland.append(midland_data)
        
        # Add unmatched Midland buildings as new records
        logger.info(f"Adding {len(unmatched_midland)} unmatched Midland buildings")
        for data in unmatched_midland:
            key = f"MD_{data['source_id'] if data['source_id'] else str(hash(data['building_name']))}"
            consolidated[key] = data
            consolidated[key]['matches'] = {}  # Initialize empty matches dict
    
    # Process Centanet data and match to existing records
    if centanet_df is not None and not centanet_df.empty:
        logger.info(f"Processing {len(centanet_df)} buildings from Centanet")
        
        # Store unmatched records for later addition
        unmatched_centanet = []
        
        for idx, row in centanet_df.iterrows():
            
            try:
                # Extract data safely with type checking
                property_id = str(row.get('propertyID')) if 'propertyID' in row else None
                building_name = str(row.get('buildingNameEn')) if 'buildingNameEn' in row and row.get('buildingNameEn') is not None else None
                address = str(row.get('address')) if 'address' in row and row.get('address') is not None else None
                district = str(row.get('districtNameEn')) if 'districtNameEn' in row and row.get('districtNameEn') is not None else None
                
                if not building_name:  # Skip records without a building name
                    continue
                    
                centanet_data = {
                    'source': 'centanet',
                    'source_id': property_id,
                    'original_key': property_id,
                    'building_name': building_name,
                    'normalized_name': normalize_building_name(building_name),
                    'address': address,
                    'district': district,
                    'original_record': row.to_dict() if hasattr(row, 'to_dict') else {},
                    'matches': {}
                }
                centanet_data = ensure_string_fields(centanet_data)
                
                # Rest of matching logic...
            except Exception as e:
                logger.warning(f"Error processing Centanet building at index {idx}: {str(e)}")
                continue

            
            # Try to match to existing buildings
            matched = False
            best_match_key = None
            best_match_score = 0
            
            for cons_key, cons_data in consolidated.items():
                match_score = calculate_similarity(centanet_data, cons_data)
                if match_score >= match_threshold and match_score > best_match_score:
                    best_match_key = cons_key
                    best_match_score = match_score
            
            if best_match_key:
                # Add as a match to existing record
                consolidated[best_match_key]['matches']['centanet'] = {
                    'id': centanet_data['source_id'],
                    'original_key': property_id,  # Store original key
                    'score': best_match_score,
                    'building_name': centanet_data['building_name'],
                    'address': centanet_data['address'],
                    'original_record': row.to_dict()  # Add original record to match info
                }
                matched = True
            
            if not matched:
                unmatched_centanet.append(centanet_data)
        
        # Add unmatched Centanet buildings as new records
        logger.info(f"Adding {len(unmatched_centanet)} unmatched Centanet buildings")
        for data in unmatched_centanet:
            key = f"CN_{data['source_id'] if data['source_id'] else str(hash(data['building_name']))}"
            consolidated[key] = data
            consolidated[key]['matches'] = {}  # Initialize empty matches dict
    
    logger.info(f"Consolidated database contains {len(consolidated)} buildings")
    return consolidated

def save_consolidated_database(building_db, output_file="consolidated_buildings.json"):
    """Save the consolidated database to both JSON and CSV formats."""
    if not building_db:
        logger.warning("Empty building database, nothing to save")
        return
        
    # Convert to DataFrame for easier manipulation
    records = []
    for key, building in building_db.items():
        building_copy = building.copy()
        building_copy['building_key'] = key
        
        # Convert matches to a string representation for CSV output
        if 'matches' in building_copy:
            building_copy['matches_json'] = json.dumps(building_copy['matches'])
            # Extract matched IDs for easier analysis
            midland_id = building_copy.get('matches', {}).get('midland', {}).get('id', '')
            centanet_id = building_copy.get('matches', {}).get('centanet', {}).get('id', '')
            building_copy['midland_id'] = midland_id
            building_copy['centanet_id'] = centanet_id
            
            # Extract original keys for verification
            midland_orig_key = building_copy.get('matches', {}).get('midland', {}).get('original_key', '')
            centanet_orig_key = building_copy.get('matches', {}).get('centanet', {}).get('original_key', '')
            building_copy['midland_original_key'] = midland_orig_key
            building_copy['centanet_original_key'] = centanet_orig_key
        
        # Remove large objects for CSV output
        if 'original_record' in building_copy:
            del building_copy['original_record']
        if 'matches' in building_copy:
            del building_copy['matches']
        
        records.append(building_copy)
    
    db_df = pd.DataFrame(records)
    
    # Save as JSON for preserving nested structures
    if output_file.endswith('.json'):
        with open(output_file, 'w', encoding='utf-8') as f:
            json.dump(records, f, ensure_ascii=False, indent=2)
    
    # Also save as CSV for easy viewing
    csv_file = output_file.replace('.json', '.csv')
    db_df.to_csv(csv_file, index=False)
    
    logger.info(f"Saved {len(records)} buildings to {output_file} and {csv_file}")
    return db_df

def generate_mapping_tables(building_db):
    """Generate mapping tables between different sources for easier reference."""
    mappings = {
        'leasinghub_to_midland': [],
        'leasinghub_to_centanet': [],
        'midland_to_centanet': []
    }
    
    for key, building in building_db.items():
        source = building.get('source')
        source_id = building.get('source_id')
        original_key = building.get('original_key')
        
        if source == 'leasinghub':
            # Map LeasingHub to Midland
            if 'midland' in building.get('matches', {}):
                midland_match = building['matches']['midland']
                mappings['leasinghub_to_midland'].append({
                    'building_key': key,
                    'leasinghub_id': source_id,
                    'leasinghub_original_key': original_key,
                    'leasinghub_name': building.get('building_name'),
                    'midland_id': midland_match.get('id'),
                    'midland_original_key': midland_match.get('original_key'),
                    'midland_name': midland_match.get('building_name'),
                    'match_score': midland_match.get('score')
                })
            
            # Map LeasingHub to Centanet
            if 'centanet' in building.get('matches', {}):
                centanet_match = building['matches']['centanet']
                mappings['leasinghub_to_centanet'].append({
                    'building_key': key,
                    'leasinghub_id': source_id,
                    'leasinghub_original_key': original_key,
                    'leasinghub_name': building.get('building_name'),
                    'centanet_id': centanet_match.get('id'),
                    'centanet_original_key': centanet_match.get('original_key'),
                    'centanet_name': centanet_match.get('building_name'),
                    'match_score': centanet_match.get('score')
                })
        
        elif source == 'midland':
            # Map Midland to Centanet
            if 'centanet' in building.get('matches', {}):
                centanet_match = building['matches']['centanet']
                mappings['midland_to_centanet'].append({
                    'building_key': key,
                    'midland_id': source_id,
                    'midland_original_key': original_key,
                    'midland_name': building.get('building_name'),
                    'centanet_id': centanet_match.get('id'),
                    'centanet_original_key': centanet_match.get('original_key'),
                    'centanet_name': centanet_match.get('building_name'),
                    'match_score': centanet_match.get('score')
                })
    
    # Convert to DataFrames and save
    for mapping_name, mapping_data in mappings.items():
        if mapping_data:
            df = pd.DataFrame(mapping_data)
            df.to_csv(f"{mapping_name}_mapping.csv", index=False)
            logger.info(f"Saved {len(df)} records to {mapping_name}_mapping.csv")
    
    return mappings

def ensure_string_fields(data_dict: Dict) -> Dict:
    """Ensure all relevant fields are strings."""
    string_fields = ['building_name', 'normalized_name', 'address', 'district']
    for field in string_fields:
        if field in data_dict:
            if data_dict[field] is None:
                data_dict[field] = ""
            else:
                data_dict[field] = str(data_dict[field])
    return data_dict

def generate_eyeball_check_report(building_db, output_file='eyeball_check_report.csv'):
    """Generate a report for eyeball checking of building matches."""
    records = []
    
    for key, building in building_db.items():
        source = building.get('source')
        base_record = {
            'building_key': key,
            'source': source,
            'source_id': building.get('source_id'),
            'original_key': building.get('original_key'),
            'building_name': building.get('building_name'),
            'address': building.get('address'),
            'district': building.get('district', ''),
            'grade': building.get('grade', ''),
            'year_built': building.get('year_built', '')
        }
        
        # If no matches, just add this record
        if not building.get('matches'):
            records.append(base_record)
            continue
        
        # For each match, create a record that shows both buildings
        for match_source, match_info in building.get('matches', {}).items():
            match_record = base_record.copy()
            match_record['match_source'] = match_source
            match_record['match_id'] = match_info.get('id')
            match_record['match_original_key'] = match_info.get('original_key')
            match_record['match_name'] = match_info.get('building_name')
            match_record['match_address'] = match_info.get('address', '')
            match_record['match_score'] = match_info.get('score')
            records.append(match_record)
    
    # Convert to DataFrame and save
    df = pd.DataFrame(records)
    df.to_csv(output_file, index=False)
    logger.info(f"Eyeball check report saved to {output_file}")
    return df

def load_source_data():
    """Load data from various sources with robust error handling."""
    leasinghub_df = None
    centanet_df = None
    midland_df = None
    
    # Load LeasingHub data if available
    if os.path.exists('leasinghub_office_buildings.csv'):
        logger.info("Loading LeasingHub data")
        leasinghub_df = load_csv_with_fallbacks('leasinghub_office_buildings.csv')
        if not leasinghub_df.empty:
            logger.info(f"Loaded {len(leasinghub_df)} buildings from LeasingHub")
    
    # Load Midland data if available
    if os.path.exists('midlandici_building_list.csv'):
        logger.info("Loading Midland data")
        midland_df = load_csv_with_fallbacks('midlandici_building_list.csv')
        if not midland_df.empty:
            logger.info(f"Loaded {len(midland_df)} buildings from Midland")
    
    # Load Centanet data with robust error handling
    if os.path.exists('centanet_ici_buildings.csv'):
        logger.info("Loading Centanet data")
        try:
            # First try the fallback loading methods
            centanet_df = load_csv_with_fallbacks('centanet_ici_buildings.csv')
            
            if centanet_df.empty:
                logger.warning("Automatic loading methods failed, preprocessing the CSV file...")
                # If all automatic methods fail, try manual preprocessing
                fixed_csv = preprocess_csv_file('centanet_ici_buildings.csv')
                centanet_df = pd.read_csv(fixed_csv)
            
            logger.info(f"Loaded {len(centanet_df)} buildings from Centanet")
        except Exception as e:
            logger.error(f"Failed to load Centanet data: {str(e)}")
    
    return leasinghub_df, centanet_df, midland_df

def process_transaction_data(building_db):
    """Process and enrich transaction data with building information."""
    # Process Centanet transactions
    if os.path.exists('centanet_transactions.csv'):
        logger.info("Processing Centanet transaction data")
        centanet_trans = load_csv_with_fallbacks('centanet_transactions.csv')
        
        if not centanet_trans.empty:
            enriched_trans = enrich_transaction_data(
                centanet_trans, 
                building_db, 
                source='centanet',
                name_col='buildingNameEn',
                id_col='propertyID'
            )
            
            enriched_trans.to_csv('enriched_centanet_transactions.csv', index=False)
            logger.info(f"Saved enriched Centanet transactions to enriched_centanet_transactions.csv")
    
    # Process Midland transactions
    if os.path.exists('midland_transactions.csv'):
        logger.info("Processing Midland transaction data")
        midland_trans = load_csv_with_fallbacks('midland_transactions.csv')
        
        if not midland_trans.empty:
            enriched_trans = enrich_transaction_data(
                midland_trans, 
                building_db, 
                source='midland',
                name_col='building_name',
                id_col='building_id'
            )
            
            enriched_trans.to_csv('enriched_midland_transactions.csv', index=False)
            logger.info(f"Saved enriched Midland transactions to enriched_midland_transactions.csv")

def enrich_transaction_data(transactions_df, building_db, source='midland', name_col='building_name', id_col=None):
    """Enrich transaction data with building information from consolidated database."""
    # Add new columns for the enriched data
    transactions_df['building_key'] = None
    transactions_df['original_building_id'] = None  # Store original ID for verification
    transactions_df['building_grade'] = None
    transactions_df['building_year'] = None
    transactions_df['match_confidence'] = None
    
    # Process each transaction
    for idx, row in transactions_df.iterrows():
        building_name = row.get(name_col) if name_col in row else None
        building_id = row.get(id_col) if id_col and id_col in row else None
        
        if not building_name and not building_id:
            continue
        
        # Try direct match by source ID first
        if building_id:
            # Search for direct match
            for key, building in building_db.items():
                # Check if this is a direct match
                if building['source'] == source and building.get('source_id') == building_id:
                    transactions_df.loc[idx, 'building_key'] = key
                    transactions_df.loc[idx, 'original_building_id'] = building.get('original_key')
                    transactions_df.loc[idx, 'building_grade'] = building.get('grade')
                    transactions_df.loc[idx, 'building_year'] = building.get('year_built')
                    transactions_df.loc[idx, 'match_confidence'] = 100
                    break
                
                # Check in matches
                if source in building.get('matches', {}):
                    match_info = building['matches'][source]
                    if match_info.get('id') == building_id:
                        transactions_df.loc[idx, 'building_key'] = key
                        transactions_df.loc[idx, 'original_building_id'] = match_info.get('original_key')
                        transactions_df.loc[idx, 'building_grade'] = building.get('grade')
                        transactions_df.loc[idx, 'building_year'] = building.get('year_built')
                        transactions_df.loc[idx, 'match_confidence'] = match_info.get('score')
                        break
        
        # If no match by ID, try fuzzy matching on name
        if pd.isna(transactions_df.loc[idx, 'building_key']) and building_name:
            norm_name = normalize_building_name(building_name)
            best_match = None
            best_score = 0
            
            for key, building in building_db.items():
                # Calculate multiple similarity metrics
                name_score = fuzz.token_sort_ratio(norm_name, building.get('normalized_name', ''))
                name_set_score = fuzz.token_set_ratio(norm_name, building.get('normalized_name', ''))
                
                best_name_score = max(name_score, name_set_score)
                
                # Check if this is better than previous matches and meets threshold
                if best_name_score > best_score and best_name_score >= 85:
                    best_match = key
                    best_score = best_name_score
            
            if best_match:
                transactions_df.loc[idx, 'building_key'] = best_match
                
                # Store original key for the appropriate source
                if building_db[best_match]['source'] == source:
                    transactions_df.loc[idx, 'original_building_id'] = building_db[best_match].get('original_key')
                elif source in building_db[best_match].get('matches', {}):
                    transactions_df.loc[idx, 'original_building_id'] = building_db[best_match]['matches'][source].get('original_key')
                
                transactions_df.loc[idx, 'building_grade'] = building_db[best_match].get('grade')
                transactions_df.loc[idx, 'building_year'] = building_db[best_match].get('year_built')
                transactions_df.loc[idx, 'match_confidence'] = best_score
    
    return transactions_df

def main():
    """Main entry point for the real estate data consolidation application."""
    logging.info("Starting real estate data consolidation process")
    
    # Step 1: Load data from various sources
    leasinghub_df, centanet_df, midland_df = load_source_data()
    
    # Step 2: Create consolidated building database
    building_db = create_consolidated_database(
        leasinghub_df=leasinghub_df,
        centanet_df=centanet_df,
        midland_df=midland_df,
        match_threshold=75  # Configurable threshold for building matching
    )
    
    # Step 3: Save consolidated database and generate mapping tables
    db_df = save_consolidated_database(building_db, 'consolidated_buildings.json')
    mappings = generate_mapping_tables(building_db)
    eyeball_df = generate_eyeball_check_report(building_db, 'eyeball_check_report.csv')
    
    # Step 4: Process transaction data (if available)
    process_transaction_data(building_db)
    
    logging.info("Real estate data consolidation process completed successfully")
    
    # Return the database and reports for further analysis in the notebook
    return building_db, db_df, mappings

# For Jupyter notebook execution
if __name__ == "__main__":
    try:
        building_db, db_df, mappings = main()
        print(f"Successfully consolidated data for {len(building_db)} buildings")
    except Exception as e:
        logging.error(f"Error in main process: {str(e)}")
        logging.exception("Stack trace:")
        print(f"Error occurred: {str(e)}")


IndentationError: expected an indented block after 'for' statement on line 353 (3588616495.py, line 356)